In [8]:
import sys
sys.path.append('/media/gpt4-pdf-chatbot-langchain/dev-langchain-ChatGLM/')
from langchain.llms.base import LLM
import torch
import transformers 
import models.shared as shared 
from abc import ABC

from langchain.llms.base import LLM
import random
from transformers.generation.logits_process import LogitsProcessor
from transformers.generation.utils import LogitsProcessorList, StoppingCriteriaList
from typing import Optional, List, Dict, Any
from models.loader import LoaderCheckPoint 
from models.base import (BaseAnswer,
                         AnswerResult)



In [2]:
import asyncio
from argparse import Namespace
from models.loader.args import parser
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
 
args = parser.parse_args(args=['--model', 'fastchat-chatglm-6b',  '--no-remote-model', '--load-in-8bit'])

args_dict = vars(args)

shared.loaderCheckPoint = LoaderCheckPoint(args_dict)
torch.cuda.empty_cache()
llm=shared.loaderLLM() 


In [3]:
import sys
sys.path.append('/media/gpt4-pdf-chatbot-langchain/dev-langchain-ChatGLM/')
from langchain.agents import Tool
from langchain.tools import BaseTool
from agent.custom_search import DeepSearch
from agent.custom_agent import *


tools = [
    Tool.from_function(
        func=DeepSearch.search,
        name="DeepSearch",
        description=""
    )
]
tool_names = [tool.name for tool in tools]
output_parser = CustomOutputParser()
prompt = CustomPromptTemplate(template=agent_template,
                              tools=tools,
                              input_variables=["related_content","tool_name", "input", "intermediate_steps"])

llm_chain = LLMChain(llm=llm, prompt=prompt)


In [4]:
llm

FastChatOpenAILLM(cache=None, verbose=False, callbacks=None, callback_manager=None, api_base_url='http://localhost:8000/v1', model_name='chatglm-6b', max_token=10000, temperature=0.01, checkPoint=<models.loader.loader.LoaderCheckPoint object at 0x7fa630590c10>, history_len=10, top_p=0.9, history=[])

In [9]:
from langchain.agents import BaseSingleActionAgent, AgentOutputParser, LLMSingleActionAgent, AgentExecutor
 

agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)
print(agent_executor.run(related_content="", input="请问近期携程有什么大的新闻", tool_name="DeepSearch"))





> Entering new AgentExecutor chain...
__call:
你现在是一个傻瓜机器人。这里是一些已知信息：



我现在有一个问题：请问近期携程有什么大的新闻

如果你知道答案，请直接给出你的回答！如果你不知道答案，请你只回答"DeepSearch('搜索词')"，并将'搜索词'替换为你认为需要搜索的关键词，除此之外不要回答其他任何内容。

下面请回答我上面提出的问题！

response:DeepSearch('旅游')
+++++++++++++++++++++++++++++++++++
DeepSearch('旅游')

Observation:震撼！恐龙展空降南山旅游景区，带你穿越侏罗纪！: 声明：该文观点仅代表作者本人，搜狐号系信息发布平台，搜狐仅提供信息存储空间服务。
承德进入旅游最佳时节 55处A级景区遍布全域: 新京报讯（记者耿子叶）6月9日，第十七届承德国际旅游文化节暨第五届承德市旅游产业发展大会正式开幕。记者在现场获悉，目前承德已经迎来旅游黄金时节，作为亚洲最受欢迎旅游城市之一，承德现有55处A级景区遍布全域。近年来，承德拓展全域全季旅游业态，今年将谋划实施文旅项目71个，总投资达到766亿元，长城国家文化公园、二十一世纪避暑山庄文化产业园、金山岭国际滑雪度假区、蟠龙湖旅游度假区、中国马镇等一批重点旅
“一带一路”电影周浦东启动！上海国际旅游度假区再迎一批重点 ...: 第二十五届上海国际电影节“一带一路”电影周今天在浦东新区举行启动仪式，一批重点影视文化企业签约落户上海国际旅游度假区，一批重点影视文创项目签约合作。 2023年是“一带一路”倡议10周年，也是“一带一路”电影周5周年。为进一步落实通过影视文化交流实践国家文化战略、推动浦东及度假区影视产业发展的要求，本届电影周活动把“文化更开放、内容更专业、形式更潮流、宣传更联动”作为主基调。 一方面凸显共建国家电
文旅大家谈｜云南旅游特色“出圈路”，路在何方？: 近段时间，一场从全国各地涌来的“进淄赶烤”热潮，引起广泛关注。淄博烧烤没什么“秘密武器”，羊肉、小饼、蘸料，配方简单，复制非常容易，但人潮还是涌向了淄博。如网友所说，没有人缺这一顿烧烤，缺的是淄博别具特色的人文环境。从消费产品到消费场景和情感，淄博烧烤透露着中国乡土社会“交情”的独特魅力。 凭借一串烧

In [7]:

from langchain.tools import StructuredTool

def multiplier(a: float, b: float) -> float:
    """Multiply the provided floats."""
    return a * b

tool = StructuredTool.from_function(multiplier)
# Structured tools are compatible with the STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION agent type. 
agent_executor = initialize_agent([tool], llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent_executor.run("What is 12111 times 14")



> Entering new AgentExecutor chain...
__call:System: Respond to the human as helpfully and accurately as possible. You have access to the following tools:

multiplier: multiplier(a: float, b: float) -> float - Multiply the provided floats., args: {{'a': {{'title': 'A', 'type': 'number'}}, 'b': {{'title': 'B', 'type': 'number'}}}}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or multiplier

Provide only ONE action per $JSON_BLOB, as shown:

```
{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{
  "action": "Final Answer",
  "action_input": "Final response to human"
}
```

Begin! Reminder to ALWAYS respond with a valid json blob

'12111'